In [85]:
import pandas as pd
from surprise import Reader, SVD, Dataset, accuracy
from surprise.model_selection import GridSearchCV, train_test_split, cross_validate

In [84]:
!pip install scikit-surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 KB 11.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp39-cp39-linux_x86_64.whl size=3193645 sha256=19d8502b68bbe05ddff6fda125e6b13908b392e2c20064269b5f6e74d8b2ce16
  Stored in directory: /root/.cache/pip/wheels/c6/3a/46/9b17b3512bdf283c6cb84f59929cdd5199d4e754d596d22784
Successfully built scikit-surprise


In [13]:
df_ratings=pd.read_csv("/content/ratings.csv")

In [14]:
df_movies=pd.read_csv("/content/movies.csv")

In [15]:
df_ratings.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,1.112486e+09
1,1,29,3.5,1.112485e+09
2,1,32,3.5,1.112485e+09
3,1,47,3.5,1.112485e+09
4,1,50,3.5,1.112485e+09


In [16]:
df_tags=pd.read_csv("/content/tags.csv")

In [17]:
df_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [18]:
df_tags.head()

,userId,movieId,tag,timestamp
0,18,4141,Mark Waters,1240597180
1,65,208,dark hero,1368150078
2,65,353,dark hero,1368150079
3,65,521,noir thriller,1368149983
4,65,592,dark hero,1368150078


In [19]:
df = df_movies.merge(df_ratings, how="left", on="movieId")
df.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.0,4.0,9.449194e+08
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,6.0,5.0,8.582755e+08
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,8.0,4.0,8.339819e+08
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,10.0,4.0,9.434979e+08
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,11.0,4.5,1.230859e+09


In [20]:
df.shape
df["title"].value_counts()
comment_counts = pd.DataFrame(df["title"].value_counts())
comment_counts.head()

,title
Pulp Fiction (1994),3881
Forrest Gump (1994),3862
"Silence of the Lambs, The (1991)",3596
"Shawshank Redemption, The (1994)",3583
Jurassic Park (1993),3452


In [38]:
rare_movies = comment_counts[comment_counts["title"] <= 1000].index

common_movies = df[~df["title"].isin(rare_movies)]

common_movies

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.0,4.0,9.449194e+08
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,6.0,5.0,8.582755e+08
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,8.0,4.0,8.339819e+08
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,10.0,4.0,9.434979e+08
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,11.0,4.5,1.230859e+09
...,...,...,...,...,...,...
1106729,58559,"Dark Knight, The (2008)",Action|Crime|Drama|IMAX,7891.0,4.5,1.370322e+09
1106730,58559,"Dark Knight, The (2008)",Action|Crime|Drama|IMAX,7892.0,4.0,1.321131e+09
1106731,58559,"Dark Knight, The (2008)",Action|Crime|Drama|IMAX,7899.0,4.0,1.328410e+09
1106732,58559,"Dark Knight, The (2008)",Action|Crime|Drama|IMAX,7904.0,2.5,1.416126e+09


In [53]:
common_movies["title"].nunique()
common_movies[common_movies["userId"]==225]

,movieId,title,genres,userId,rating,timestamp
9942,11,"American President, The (1995)",Comedy|Drama|Romance,225.0,5.0,838831682.0
14375,19,Ace Ventura: When Nature Calls (1995),Comedy,225.0,1.0,838831543.0
23777,34,Babe (1995),Children|Drama,225.0,3.0,838831507.0
27000,39,Clueless (1995),Comedy|Romance,225.0,3.0,838831573.0
30508,47,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,225.0,3.0,838831507.0
33752,50,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,225.0,3.0,838831573.0
48529,110,Braveheart (1995),Action|Drama|War,225.0,1.0,838831479.0
58113,150,Apollo 13 (1995),Adventure|Drama|IMAX,225.0,5.0,838831339.0
65807,161,Crimson Tide (1995),Drama|Thriller|War,225.0,5.0,838831416.0
68561,165,Die Hard: With a Vengeance (1995),Action|Crime|Thriller,225.0,5.0,838831369.0


In [23]:
user_movie_df = common_movies.pivot_table(index=["userId"], columns=["title"], values="rating")

user_movie_df.shape

(7779, 201)

In [24]:
user_movie_df.head(10)

title,2001: A Space Odyssey (1968),"Abyss, The (1989)",Ace Ventura: Pet Detective (1994),Ace Ventura: When Nature Calls (1995),Addams Family Values (1993),Airplane! (1980),Aladdin (1992),Alien (1979),Aliens (1986),Almost Famous (2000),...,Twister (1996),"Usual Suspects, The (1995)",Waterworld (1995),What's Eating Gilbert Grape (1993),When Harry Met Sally... (1989),While You Were Sleeping (1995),Who Framed Roger Rabbit? (1988),Willy Wonka & the Chocolate Factory (1971),"Wizard of Oz, The (1939)",X-Men (2000)
userId,,,,,,,,,,,,,,,,,,,,,
1.0,3.5,NaN,NaN,NaN,NaN,NaN,NaN,4.0,4.0,NaN,...,NaN,3.5,NaN,3.5,NaN,NaN,NaN,NaN,3.5,NaN
2.0,5.0,NaN,NaN,NaN,NaN,2.0,NaN,5.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3.0,5.0,3.0,NaN,NaN,NaN,5.0,NaN,5.0,4.0,NaN,...,NaN,5.0,NaN,3.0,4.0,NaN,NaN,5.0,4.0,NaN
4.0,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5.0,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,...,5.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN
6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
7.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.0,NaN,NaN,NaN,3.0,3.0,4.0,NaN,NaN,4.0
8.0,NaN,NaN,3.0,1.0,NaN,NaN,3.0,NaN,NaN,NaN,...,NaN,NaN,3.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN
9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

Item Based Recommendation

In [67]:
random_user = int(pd.Series(user_movie_df.index).sample(1, random_state=50).values)
random_user
random_user_df = user_movie_df[user_movie_df.index == random_user]
random_user_df

title,2001: A Space Odyssey (1968),"Abyss, The (1989)",Ace Ventura: Pet Detective (1994),Ace Ventura: When Nature Calls (1995),Addams Family Values (1993),Airplane! (1980),Aladdin (1992),Alien (1979),Aliens (1986),Almost Famous (2000),...,Twister (1996),"Usual Suspects, The (1995)",Waterworld (1995),What's Eating Gilbert Grape (1993),When Harry Met Sally... (1989),While You Were Sleeping (1995),Who Framed Roger Rabbit? (1988),Willy Wonka & the Chocolate Factory (1971),"Wizard of Oz, The (1939)",X-Men (2000)
userId,,,,,,,,,,,,,,,,,,,,,
5675.0,NaN,NaN,3.5,NaN,NaN,NaN,4.0,NaN,NaN,NaN,...,3.0,5.0,NaN,NaN,NaN,NaN,3.0,NaN,NaN,3.5


In [73]:
recently_5rated_movie_id = df_ratings[(df_ratings['userId']==random_user)&(df_ratings['rating']==5)].sort_values(by='timestamp', ascending=False)['movieId'].values[0]
recently_5rated_movie_id

50

In [75]:
recently_5rated_movie_title=common_movies[common_movies['movieId']==recently_5rated_movie_id]['title'].values[0]
recently_5rated_movie_title

'Usual Suspects, The (1995)'

In [76]:
user_movie_df_filtered = user_movie_df[recently_5rated_movie_title]
user_movie_df_filtered

userId
1.0       3.5
2.0       NaN
3.0       5.0
4.0       NaN
5.0       NaN
         ... 
7904.0    3.5
7905.0    NaN
7906.0    NaN
7907.0    NaN
7908.0    NaN
Name: Usual Suspects, The (1995), Length: 7779, dtype: float64

Finding correlation between selected movie and remaining movies using filtered df and sorting them

In [77]:
user_movie_df.corrwith(user_movie_df_filtered).sort_values(ascending=False)

title
Usual Suspects, The (1995)        1.000000
L.A. Confidential (1997)          0.390854
American History X (1998)         0.319381
Seven (a.k.a. Se7en) (1995)       0.313349
Memento (2000)                    0.308962
                                    ...   
Mr. Holland's Opus (1995)         0.058546
Star Trek: Generations (1994)     0.054054
American President, The (1995)    0.044716
Titanic (1997)                    0.021758
Net, The (1995)                   0.015023
Length: 201, dtype: float64

selecting top 5 movies similar to the filtered movie

In [78]:
user_movie_df.corrwith(user_movie_df_filtered).sort_values(ascending=False)[1:6]

title
L.A. Confidential (1997)       0.390854
American History X (1998)      0.319381
Seven (a.k.a. Se7en) (1995)    0.313349
Memento (2000)                 0.308962
Gladiator (2000)               0.295285
dtype: float64

Taking a random sample of movie id's for model based recommendation

In [79]:
movie_ids = [541, 147, 1909, 59103] # ids of selected movies
movies = ["Blade Runner (1982)",
          "Basketball Diaries, The (1995)",
          "X-Files: Fight the Future, The (1998)",
          "Forbidden Kingdom, The (2008)"]

In [82]:
sample_df = df[df.movieId.isin(movie_ids)]
sample_df.head()


,movieId,title,genres,userId,rating,timestamp
57706,147,"Basketball Diaries, The (1995)",Drama,29.0,3.0,8.355634e+08
57707,147,"Basketball Diaries, The (1995)",Drama,61.0,4.0,1.197164e+09
57708,147,"Basketball Diaries, The (1995)",Drama,69.0,4.0,8.668167e+08
57709,147,"Basketball Diaries, The (1995)",Drama,101.0,4.0,8.363262e+08
57710,147,"Basketball Diaries, The (1995)",Drama,134.0,2.0,9.450658e+08


In [86]:
user_movie_df_1 = sample_df.pivot_table(index=["userId"],
                                      columns=["title"],
                                      values="rating")
user_movie_df_1

title,"Basketball Diaries, The (1995)",Blade Runner (1982),"Forbidden Kingdom, The (2008)","X-Files: Fight the Future, The (1998)"
userId,,,,
1.0,NaN,4.0,NaN,NaN
2.0,NaN,5.0,NaN,NaN
3.0,NaN,5.0,NaN,5.0
11.0,NaN,4.5,NaN,NaN
21.0,NaN,5.0,NaN,NaN
...,...,...,...,...
7880.0,NaN,5.0,NaN,NaN
7881.0,NaN,NaN,NaN,2.0
7882.0,NaN,4.5,NaN,NaN


In [87]:
reader = Reader(rating_scale=(1, 5)) 

In [88]:
data = Dataset.load_from_df(sample_df[['userId',
                                       'movieId',
                                       'rating']], reader)

In [89]:
trainset, testset = train_test_split(data, test_size=.25)

In [90]:
svd_model = SVD()

In [91]:
svd_model.fit(trainset)

# Test the model with the testing set using p and q values
predictions = svd_model.test(testset)

In [92]:
predictions

[Prediction(uid=140.0, iid=541, r_ui=5.0, est=4.140836967282436, details={'was_impossible': False}),
 Prediction(uid=5444.0, iid=541, r_ui=5.0, est=4.140836967282436, details={'was_impossible': False}),
 Prediction(uid=2024.0, iid=1909, r_ui=3.5, est=3.782680370484612, details={'was_impossible': False}),
 Prediction(uid=6306.0, iid=541, r_ui=4.5, est=4.140836967282436, details={'was_impossible': False}),
 Prediction(uid=2406.0, iid=541, r_ui=5.0, est=4.140836967282436, details={'was_impossible': False}),
 Prediction(uid=2225.0, iid=1909, r_ui=5.0, est=3.360200214871157, details={'was_impossible': False}),
 Prediction(uid=1166.0, iid=541, r_ui=4.5, est=4.140836967282436, details={'was_impossible': False}),
 Prediction(uid=6127.0, iid=541, r_ui=4.0, est=4.140836967282436, details={'was_impossible': False}),
 Prediction(uid=2033.0, iid=541, r_ui=3.5, est=4.140836967282436, details={'was_impossible': False}),
 Prediction(uid=630.0, iid=1909, r_ui=3.0, est=3.3370580697220613, details={'was_

In [93]:
svd_model.predict(uid=1.0, iid=541, verbose=True) # the estimated rating value by the model for the user with the userId=1

sample_df[sample_df["userId"] == 1.0] 

user: 1.0        item: 541        r_ui = None   est = 3.93   {'was_impossible': False}


,movieId,title,genres,userId,rating,timestamp
209266,541,Blade Runner (1982),Action|Sci-Fi|Thriller,1.0,4.0,1.112485e+09


In [94]:
accuracy.rmse(predictions)

RMSE: 0.9225


0.9224526357928591